In [ ]:
# ===============================
# PHASE 1: DATA LOADING & UNDERSTANDING
# ===============================

import pandas as pd

# Load dataset
df = pd.read_csv("Project_1Data.csv")

# Basic inspection
print("Shape:", df.shape)
print("Columns:", list(df.columns))

# Missing values & duplicates (GOOD PRACTICE)
missing_values = df.isnull().sum()
duplicate_rows = df.duplicated().sum()

print("\nMissing values per column:\n", missing_values)
print("\nDuplicate rows:", duplicate_rows)


# ===============================
# PHASE 2: DATA CLEANING & PREPARATION
# ===============================

# Convert OrderDate to datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Fix numeric data types
df['Quantity'] = df['Quantity'].astype(int)
df['Price'] = df['Price'].astype(float)
df['Revenue'] = df['Revenue'].astype(float)

# Revenue sanity check
df['CalculatedRevenue'] = df['Quantity'] * df['Price']

# Validate revenue correctness
revenue_check = (df['Revenue'] == df['CalculatedRevenue']).value_counts()
print("\nRevenue validation:\n", revenue_check)

# Correct revenue if needed
df['Revenue'] = df['CalculatedRevenue']
df.drop(columns=['CalculatedRevenue'], inplace=True)

# Remove duplicates & missing values (safety)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)


# ===============================
# PHASE 3: EXPLORATORY DATA ANALYSIS
# ===============================

# ---- Business Snapshot ----
total_orders = df['OrderID'].nunique()
total_revenue = df['Revenue'].sum()
aov = total_revenue / total_orders

# ---- Monthly Revenue Trend ----
df['MonthName'] = df['OrderDate'].dt.month_name()

monthly_revenue = (
    df.groupby('MonthName')['Revenue']
    .sum()
    .sort_values(ascending=False)
)

best_month = monthly_revenue.idxmax()
worst_month = monthly_revenue.idxmin()

# ---- City-wise Performance ----
city_revenue = df.groupby('City')['Revenue'].sum().sort_values(ascending=False)
top_3_cities = city_revenue.head(3)
lowest_city = city_revenue.idxmin()

# ---- Product Performance ----
product_revenue = df.groupby('Product')['Revenue'].sum().sort_values(ascending=False)
top_products = product_revenue.head(5)
bottom_products = product_revenue.tail(5)

# High quantity but low revenue products
product_analysis = df.groupby('Product').agg(
    TotalQuantity=('Quantity', 'sum'),
    TotalRevenue=('Revenue', 'sum')
)

high_qty_low_revenue = product_analysis.sort_values(
    by=['TotalQuantity', 'TotalRevenue'],
    ascending=[False, True]
).head(5)

# ---- Category-wise Analysis ----
category_analysis = df.groupby('Category').agg(
    TotalRevenue=('Revenue', 'sum'),
    Orders=('OrderID', 'nunique')
)

category_analysis['AOV'] = category_analysis['TotalRevenue'] / category_analysis['Orders']
most_profitable_category = category_analysis['TotalRevenue'].idxmax()

# ---- Order Value Distribution ----
order_values = df.groupby('OrderID')['Revenue'].sum()

min_order = order_values.min()
max_order = order_values.max()
mean_order = order_values.mean()
median_order = order_values.median()


# ===============================
# PHASE 4: BUSINESS INSIGHTS & RECOMMENDATIONS
# ===============================

"""
TASK 4.1 – EXECUTIVE SUMMARY

The analysis covers approximately 500 retail orders generated during 2023.
The business achieved a total revenue of ₹{total_revenue:,.0f} across {total_orders} orders,
resulting in an average order value (AOV) of approximately ₹{aov:,.0f}.
Overall performance indicates stable sales with opportunities for revenue optimization
through product and regional strategies.
"""

"""
TASK 4.2 – REVENUE TREND INSIGHT

Month-wise revenue analysis shows an overall positive trend with moderate fluctuations.
The highest revenue was recorded in {best_month}, likely due to increased consumer demand.
The lowest revenue occurred in {worst_month}, possibly due to seasonal slowdown.
Overall, the business demonstrates consistent revenue generation with seasonal variation.
"""

"""
TASK 4.3 – CITY-WISE BUSINESS INSIGHT

The top three cities contribute a significant portion of total revenue,
indicating strong regional demand concentration.
The lowest contributing city may require targeted marketing efforts or cost optimization.
Expanding high-performing cities could further improve overall revenue.
"""

"""
TASK 4.4 – PRODUCT STRATEGY INSIGHT

A small number of products generate the majority of revenue, highlighting key revenue drivers.
Several products show low revenue contribution and may require pricing or promotional review.
Products with high sales volume but low revenue suggest possible underpricing
or bundling opportunities.
"""

"""
TASK 4.5 – CATEGORY-LEVEL DECISION MAKING

The most profitable category contributes the highest share of revenue.
Categories with higher AOV indicate better customer spending behavior.
Lower-performing categories may benefit from discount optimization or inventory review.
"""

"""
TASK 4.6 – RISK & OPPORTUNITY SUMMARY

Risks:
1. Revenue dependency on a limited number of products.
2. Sales concentration in a few cities increases regional risk.

Opportunities:
1. Upselling high-volume low-revenue products.
2. Expanding operations in top-performing cities.
"""

"""
TASK 4.7 – FINAL BUSINESS RECOMMENDATIONS

1. Increase marketing investment in top-performing cities.
2. Re-evaluate pricing for high-volume, low-revenue products.
3. Focus inventory expansion on the most profitable categories.
4. Improve performance in low-revenue months through targeted campaigns.
5. Reduce dependency on a limited number of revenue-generating products.
"""
